## Wisconsin Diagnostic Breast Cancer (WDBC) Dataset

In [1]:
# load modules
import pandas as pd
from src import read
from src import preprocessing
from src import eda
from src import modelling
from src import experimental

In [2]:
# define data folder path and data file names
raw_data_folder = r"C:\Users\35799\Desktop\cookiecutter-analytical-project\biolizard-internship-marios\data\raw"
datafile_name = r"wdbc.data"

# load data
data_df = read.data_load(folder=raw_data_folder, filename=datafile_name, delimiter=',')

# create column labels
column_labels = ["F"+str(i+1) for i in range(data_df.shape[1]-2)]
column_labels.insert(0, "Diagnosis")
column_labels.insert(0, "ID")
data_df.columns = column_labels


### Data Info

In [3]:
data_df.head()

,ID,Diagnosis,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30
0,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
1,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
2,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
3,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
4,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.007510,0.03345,0.03672,0.01137,0.02165,0.005082,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


In [4]:
id_features, categorical_features, continuous_features = read.data_info(data_df, threshold=20)

DIMENSIONS:
----------------------------------------------------------------------------------------------------
Entries: 568
Features: 32
----------------------------------------------------------------------------------------------------


CATEGORICAL FEATURES:
----------------------------------------------------------------------------------------------------
Features    Data Type    Categories & Counts
----------  -----------  ---------------------
Diagnosis   object       {'B': 357, 'M': 211}
----------------------------------------------------------------------------------------------------


CONTINUOUS FEATURES:
----------------------------------------------------------------------------------------------------
Features    Data Type      Count     Mean      Std      Min     25th    Median      75th       Max
----------  -----------  -------  -------  -------  -------  -------  --------  --------  --------
F1          float64          568   14.12     3.52     6.981   11.698    13

### Data Preprocessing

#### Data Splitting

In [5]:
X, y, X_train, y_train, X_test, y_test, _, _ = preprocessing.data_split(data_df, target="Diagnosis", method="tt", train_proportion=0.8, stratify=True, random_state=0)

#### Missing Values

In [6]:
train_df, test_df = preprocessing.treat_nan(X_train, y_train, X_test, y_test, id_features, categorical_features, continuous_features, target="Diagnosis", drop_nan_rows=False, impute_cutoff=0.5, categorical_imputer="mode", continuous_imputer="median")

#### Automatic Outlier Detection

In [7]:
train_df, test_df = preprocessing.treat_outliers(train_df, test_df, id_features, categorical_features, continuous_features, target="Diagnosis", method="if", outlier_fraction=0.1)

The following entries are probable outliers as identified by the Isolation Forest technique (train set):


,ID,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,Diagnosis
563,926424,21.560,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.700,0.010300,0.028910,0.05198,0.024540,0.011140,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.41070,0.22160,0.2060,0.07115,M
351,899987,25.730,17.46,174.20,2010.0,0.11490,0.23630,0.33680,0.19130,0.1956,0.06121,0.9948,0.8509,7.222,153.100,0.006369,0.042430,0.04266,0.015080,0.023350,0.003385,33.130,23.58,229.30,3234.0,0.15300,0.59370,0.64510,0.27560,0.3690,0.08815,M
350,899667,15.750,19.22,107.10,758.6,0.12430,0.23640,0.29140,0.12420,0.2375,0.07603,0.5204,1.3240,3.477,51.220,0.009329,0.065590,0.09953,0.022830,0.055430,0.007330,17.360,24.17,119.40,915.3,0.15500,0.50460,0.68720,0.21350,0.4245,0.10500,M
399,90439701,17.910,21.02,124.40,994.0,0.12300,0.25760,0.31890,0.11980,0.2113,0.07115,0.4030,0.7747,3.123,41.510,0.007159,0.037180,0.06165,0.010510,0.015910,0.005099,20.800,27.78,149.60,1304.0,0.18730,0.59170,0.90340,0.19640,0.3245,0.11980,M
107,86355,22.270,19.67,152.80,1509.0,0.13260,0.27680,0.42640,0.18230,0.2556,0.07039,1.2150,1.5450,10.050,170.000,0.006515,0.086680,0.10400,0.024800,0.031120,0.005037,28.400,28.01,206.80,2360.0,0.17010,0.69970,0.96080,0.29100,0.4055,0.09789,M
271,8910988,21.750,20.99,147.30,1491.0,0.09401,0.19610,0.21950,0.10880,0.1721,0.06194,1.1670,1.3520,8.867,156.800,0.005687,0.049600,0.06329,0.015610,0.019240,0.004614,28.190,28.18,195.90,2384.0,0.12720,0.47250,0.58070,0.18410,0.2833,0.08858,M
502,915143,23.090,19.83,152.10,1682.0,0.09342,0.12750,0.16760,0.10030,0.1505,0.05484,1.2910,0.7452,9.635,180.200,0.005753,0.033560,0.03976,0.021560,0.022010,0.002897,30.790,23.87,211.50,2782.0,0.11990,0.36250,0.37940,0.22640,0.2908,0.07277,M
255,88649001,19.550,28.77,133.60,1207.0,0.09260,0.20630,0.17840,0.11440,0.1893,0.06232,0.8426,1.1990,7.158,106.400,0.006356,0.047650,0.03863,0.015190,0.019360,0.005252,25.050,36.27,178.60,1926.0,0.12810,0.53290,0.42510,0.19410,0.2818,0.10050,M
317,894329,9.042,18.90,60.07,244.5,0.09968,0.19720,0.19750,0.04908,0.2330,0.08743,0.4653,1.9110,3.769,24.200,0.009845,0.065900,0.10270,0.025270,0.034910,0.007877,10.060,23.40,68.62,297.1,0.12210,0.37480,0.46090,0.11450,0.3135,0.10550,B
2,84348301,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.230,0.009110,0.074580,0.05661,0.018670,0.059630,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.68690,0.25750,0.6638,0.17300,M


The following entries are probable outliers as identified by the Isolation Forest technique (test set):


,ID,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,Diagnosis
137,868826,14.950,17.57,96.85,678.1,0.11670,0.13050,0.1539,0.08624,0.1957,0.06216,1.2960,1.4520,8.419,101.90,0.010000,0.034800,0.06577,0.02801,0.05168,0.002887,18.55,21.43,121.40,971.4,0.14110,0.21640,0.3355,0.1667,0.3414,0.07147,M
82,8611792,19.100,26.29,129.10,1132.0,0.12150,0.17910,0.1937,0.14690,0.1634,0.07224,0.5190,2.9100,5.801,67.10,0.007545,0.060500,0.02134,0.01843,0.03056,0.010390,20.33,32.72,141.30,1298.0,0.13920,0.28170,0.2432,0.1841,0.2311,0.09203,M
378,9013838,11.080,18.83,73.30,361.6,0.12160,0.21540,0.1689,0.06367,0.2196,0.07950,0.2114,1.0270,1.719,13.99,0.007405,0.045490,0.04588,0.01339,0.01738,0.004435,13.24,32.82,91.76,508.1,0.21840,0.93790,0.8402,0.2524,0.4154,0.14030,M
251,885429,19.730,19.82,130.70,1206.0,0.10620,0.18490,0.2417,0.09740,0.1733,0.06697,0.7661,0.7800,4.115,92.81,0.008482,0.050570,0.06800,0.01971,0.01467,0.007259,25.28,25.59,159.80,1933.0,0.17100,0.59550,0.8489,0.2507,0.2749,0.12970,M
504,915276,9.676,13.14,64.12,272.5,0.12550,0.22040,0.1188,0.07038,0.2057,0.09575,0.2744,1.3900,1.787,17.67,0.021770,0.048880,0.05189,0.01450,0.02632,0.011480,10.60,18.04,69.47,328.1,0.20060,0.36630,0.2913,0.1075,0.2848,0.13640,B
77,8610862,20.180,23.97,143.70,1245.0,0.12860,0.34540,0.3754,0.16040,0.2906,0.08142,0.9317,1.8850,8.649,116.40,0.010380,0.068350,0.10910,0.02593,0.07895,0.005987,23.37,31.72,170.30,1623.0,0.16390,0.61640,0.7681,0.2508,0.5440,0.09964,M
338,89812,23.510,24.27,155.10,1747.0,0.10690,0.12830,0.2308,0.14100,0.1797,0.05506,1.0090,0.9245,6.462,164.10,0.006292,0.019710,0.03582,0.01301,0.01479,0.003118,30.67,30.73,202.40,2906.0,0.15150,0.26780,0.4819,0.2089,0.2593,0.07738,M
235,88299702,23.210,26.97,153.50,1670.0,0.09509,0.16820,0.1950,0.12370,0.1909,0.06309,1.0580,0.9635,7.247,155.80,0.006428,0.028630,0.04497,0.01716,0.01590,0.003053,31.01,34.51,206.00,2944.0,0.14810,0.41260,0.5820,0.2593,0.3103,0.08677,M
8,84501001,12.460,24.04,83.97,475.9,0.11860,0.23960,0.2273,0.08543,0.2030,0.08243,0.2976,1.5990,2.039,23.94,0.007149,0.072170,0.07743,0.01432,0.01789,0.010080,15.09,40.68,97.65,711.4,0.18530,1.05800,1.1050,0.2210,0.4366,0.20750,M
560,925311,11.200,29.37,70.67,386.0,0.07449,0.03558,0.0000,0.00000,0.1060,0.05502,0.3141,3.8960,2.041,22.81,0.007594,0.008878,0.00000,0.00000,0.01989,0.001773,11.92,38.30,75.19,439.6,0.09267,0.05494,0.0000,0.0000,0.1566,0.05905,B


#### Target Feature Balance

In [8]:
preprocessing.target_balance_check(train_df, target="Diagnosis", imbalance_fraction=0.3, graphic=False)

Target Feature Levels      Counts    Percentages (%)
-----------------------  --------  -----------------
B                             272               66.7
M                             136               33.3


The tagret feature levels are unbalanced.


In [9]:
# sampling techniques to treat target feature imbalance

# oversampling ratios must be greater or equal to 1 and are based on the class with the most counts
oversampling_ratios = {"B": 1, "M": 1}
train_df_oversampled = preprocessing.sampler(train_df, target="Diagnosis", method="over", sampling_ratios=oversampling_ratios, random_state=0, graphic=False)

Balanced target feature (oversampling):
Target Feature Levels      Counts    Percentages (%)
-----------------------  --------  -----------------
B                             272                 50
M                             272                 50


In [10]:
# undersampling ratios must be less or equal to 1 and are based on the class with the least counts
undersampling_ratios = {"B": 1, "M": 1}
train_df_undersampled = preprocessing.sampler(train_df, target="Diagnosis", method="under", sampling_ratios=undersampling_ratios, random_state=0, graphic=False)

Balanced target feature (undersampling):
Target Feature Levels      Counts    Percentages (%)
-----------------------  --------  -----------------
B                             136                 50
M                             136                 50


### Data Exploration

In [11]:
# data sample for examples
sample_df = train_df.iloc[:, 1:11]
sample_df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
380,11.04,14.93,70.67,372.7,0.07987,0.07079,0.03546,0.020740,0.2003,0.06246
143,10.75,14.97,68.26,355.3,0.07793,0.05139,0.02251,0.007875,0.1399,0.05688
135,11.71,16.67,74.72,423.6,0.10510,0.06095,0.03592,0.026000,0.1339,0.05945
115,8.95,15.76,58.74,245.2,0.09462,0.12430,0.09263,0.023080,0.1305,0.07163
34,16.74,21.59,110.10,869.5,0.09610,0.13360,0.13480,0.060180,0.1896,0.05656


#### Correlation Matrix & Heatmap

In [12]:
# correlation matrix
pearson_correlation_matrix = eda.correlations(sample_df, type="pearson", printout="matrix")

Pearson Correlation Matrix:


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10
F1,1.000,0.247,0.998,0.992,0.118,0.495,0.685,0.795,0.118,-0.390
F2,0.247,1.000,0.251,0.253,-0.036,0.170,0.257,0.222,0.068,-0.092
F3,0.998,0.251,1.000,0.990,0.154,0.541,0.721,0.823,0.145,-0.349
F4,0.992,0.253,0.990,1.000,0.117,0.482,0.684,0.796,0.123,-0.377
F5,0.118,-0.036,0.154,0.117,1.000,0.669,0.512,0.538,0.511,0.597
F6,0.495,0.170,0.541,0.482,0.669,1.000,0.887,0.831,0.502,0.471
F7,0.685,0.257,0.721,0.684,0.512,0.887,1.000,0.940,0.396,0.208
F8,0.795,0.222,0.823,0.796,0.538,0.831,0.940,1.000,0.401,0.088
F9,0.118,0.068,0.145,0.123,0.511,0.502,0.396,0.401,1.000,0.354
F10,-0.390,-0.092,-0.349,-0.377,0.597,0.471,0.208,0.088,0.354,1.000


In [13]:
# correlation heatmap
spearman_correlation_matrix = eda.correlations(sample_df, type="spearman", printout="heatmap")

#### Boxplots

In [14]:
eda.box_plot(train_df, features=continuous_features, stratify_var="Diagnosis", group_var="", title="Boxplot of ", xtitle="Diagnosis", ytitle="Feature value", widget_description="Select: ", stratify=True, group=False)

In [15]:
eda.box_subplots(train_df, features=list(sample_df.columns), stratify_var="Diagnosis", columns=5, width=250, height=250, stratify=True)

#### Histograms

In [16]:
eda.hist_plot(train_df, features=list(sample_df.columns), group_var="Diagnosis", title="Histogram of ", xtitle="Bins", ytitle="Count", widget_description="Select: ", group=True)

In [17]:
eda.hist_subplots(sample_df, columns=5, width=250, height=250)

#### Distribution plots

In [18]:
eda.dist_plot(sample_df.iloc[:, 0:1], title="Distribution Plot:")

In [19]:
eda.dist_subplots(sample_df, columns=5, width=250, height=250)

#### Dimensionality Reduction

##### PCA

In [20]:
# Principal Component Analysis
eda.pca_variance_plot(train_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis")

In [21]:
_, pca2d_train_df, _, pca2d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="pca", plot_type="2d", components=2)

In [22]:
_, pca3d_train_df, _, pca3d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="pca", plot_type="3d", components=3)

In [23]:
_, pca5d_train_df, _, pca5d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="pca", plot_type="multi", components=5)

##### MDS

In [24]:
# _, mds2d_train_df, _, mds2d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="mds", plot_type="2d", components=2)

In [25]:
# _, mds3d_train_df, _, mds3d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="mds", plot_type="3d", components=3)

In [26]:
# _, mds5d_train_df, _, mds5d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="mds", plot_type="multi", components=5)

##### t-SNE

In [27]:
# _, tsne2d_train_df, _, tsne2d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="tsne", plot_type="2d", components=2, perplexity=30)

In [28]:
# _, tsne3d_train_df, _, tsne3d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="tsne", plot_type="3d", components=3, perplexity=30)

##### UMAP

In [29]:
# _, umpa2d_train_df, _, umpa2d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="umap", plot_type="2d", components=2, neighbors=100, min_distance=0.1, metric="euclidean")

In [30]:
# _, umpa3d_train_df, _, umpa3d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="umap", plot_type="3d", components=3, neighbors=100, min_distance=0.1, metric="euclidean")

In [31]:
# _, umpa5d_train_df, _, umpa5d_test_df = eda.dimensionality_reduction(train_df, test_df, identifier=id_features, categorical=categorical_features, continuous=continuous_features, target="Diagnosis", method="umap", plot_type="multi", components=5, neighbors=100, min_distance=0.1, metric="euclidean")

#### Cluster Analysis

In [32]:
pca2d_train_df.head()

,ID,PC1,PC2,Diagnosis
0,901836,-3.275536,0.221306,B
1,869254,-4.363844,-0.924405,B
2,868223,-2.196172,-0.119097,B
3,864726,-1.410305,5.456747,B
4,854253,4.281450,-0.282671,M


##### K-Means

In [33]:
# eda.kmeans_inertia_plot(pca2d_train_df, identifier=id_features, target="Diagnosis")

In [34]:
kmeans2d_train_df = eda.clustering(df=train_df, input_df=pca2d_train_df, identifier=id_features, target="Diagnosis", method="kmeans", plot_type="2d", marker_size_ref="F1", clusters=2, initializations=100, random_state=0)

In [35]:
kmeans3d_train_df = eda.clustering(df=train_df, input_df=pca3d_train_df, identifier=id_features, target="Diagnosis", method="kmeans", plot_type="3d", marker_size_ref="F1", clusters=2, initializations=100, random_state=0)

##### Hierarchical

In [36]:
# hierarchical2d_train_df = eda.clustering(df=train_df, input_df=pca2d_train_df, identifier=id_features, target="Diagnosis", method="hierarchical", plot_type="2d", marker_size_ref="F1", clusters=2)

In [37]:
# hierarchical3d_train_df = eda.clustering(df=train_df, input_df=pca3d_train_df, identifier=id_features, target="Diagnosis", method="hierarchical", plot_type="3d", marker_size_ref="F1", clusters=2)

##### DBSCAN

In [38]:
# dbscan2d_train_df = eda.clustering(df=train_df, input_df=pca2d_train_df, identifier=id_features, target="Diagnosis", method="dbscan", plot_type="2d", marker_size_ref="F1", epsilon=1, minimum_samples=10)

In [39]:
# dbscan3d_train_df = eda.clustering(df=train_df, input_df=pca3d_train_df, identifier=id_features, target="Diagnosis", method="dbscan", plot_type="3d", marker_size_ref="F1", epsilon=1, minimum_samples=15)

### Data Modelling

#### Feature Selection

##### Recursive Feature Ellimination

In [40]:
# works with estimators "Tree", "Logistic", "linearSVC", "Forest" and "AdaBoost"
# "SVC", "Bagging" and "KNN" estimators do not work
rfe_df = modelling.feature_selection(train_df, identifier=id_features, target="Diagnosis", method="RFE", estimator="Tree", selected_features_number=None)

Initial dataset dimensions:
Rows: 408
Columns: 32


Dataset dimensions after feature selection:
Rows: 408
Columns: 17


In [42]:
# works only with estimator "Forest"
boruta_df = modelling.feature_selection(train_df, identifier=id_features, target="Diagnosis", method="Boruta", estimator="Forest", random_state=0)

Initial dataset dimensions:
Rows: 408
Columns: 32


Dataset dimensions after feature selection:
Rows: 408
Columns: 24


In [43]:
# works only with estimators "Logistic" and "linearSVC"
l1_df = modelling.feature_selection(train_df, identifier=id_features, target="Diagnosis", method="L1", estimator="linearSVC")

Initial dataset dimensions:
Rows: 408
Columns: 32


Dataset dimensions after feature selection:
Rows: 408
Columns: 15


#### Model Fitting

In [65]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [66]:
label_dict = {"B": 0, "M": 1}

X_train = rfe_df.drop(columns=['ID', 'Diagnosis'])
y_train = rfe_df[['Diagnosis']].replace(label_dict)

X_test = test_df[X_train.columns]
y_test = test_df[['Diagnosis']].replace(label_dict)


In [67]:
models = []
models.append(['Logistic Regression', LogisticRegression(random_state=0)])
models.append(['SVM', SVC(random_state=0)])
models.append(['Linear SVM', LinearSVC(random_state=0)])
models.append(['SGD', SGDClassifier(random_state=0)])
models.append(['K-nearest Neigbors', KNeighborsClassifier()])
models.append(['GaussianNB', GaussianNB()])
models.append(['Decision Tree', DecisionTreeClassifier(random_state=0)])
models.append(['Bagging', BaggingClassifier(random_state=0)])
models.append(['Random Forest', RandomForestClassifier(random_state=0)])
models.append(['AdaBoost', AdaBoostClassifier(random_state=0)])
models.append(['XGBoost', GradientBoostingClassifier(random_state=0)])
models.append(['MLP', MLPClassifier(random_state=0)])


In [84]:
lst_1 = []

for i, m in enumerate(tqdm(range(len(models)))):
    lst_2 = []
    model = models[m][1]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(cm, index=sorted(train_df["Diagnosis"].unique()), columns=sorted(train_df["Diagnosis"].unique()))
    accuracies = cross_val_score(estimator=model, X=X_train, y=y_train, cv=10)

    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f'{i}) {models[m][0]}:')
    print('-'*len(models[m][0]))
    print("Confusion Matrix:")
    print(cm_df)
    print(f'Accuracy: {round(accuracy, 3)}')
    print(f'Recall: {round(recall, 3)}')
    print(f'Presicion: {round(precision, 3)}')
    print('-'*100)
   
    lst_2.append(models[m][0])
    lst_2.append(cm_df.iloc[0][0])
    lst_2.append(cm_df.iloc[0][1])
    lst_2.append(cm_df.iloc[1][0])
    lst_2.append(cm_df.iloc[1][1])
    lst_2.append(accuracy)
    lst_2.append(recall)
    lst_2.append(precision)
    lst_1.append(lst_2)

  8%|▊         | 1/12 [00:00<00:03,  2.99it/s]

0) Logistic Regression:
-------------------
Confusion Matrix:
    B   M
B  68   2
M   4  28
Accuracy: 0.941
Recall: 0.875
Presicion: 0.933
----------------------------------------------------------------------------------------------------
1) SVM:
---
Confusion Matrix:
    B   M
B  69   1
M   6  26
Accuracy: 0.931
Recall: 0.812
Presicion: 0.963
----------------------------------------------------------------------------------------------------


 42%|████▏     | 5/12 [00:00<00:00,  9.01it/s]

2) Linear SVM:
----------
Confusion Matrix:
    B   M
B  66   4
M   6  26
Accuracy: 0.902
Recall: 0.812
Presicion: 0.867
----------------------------------------------------------------------------------------------------
3) SGD:
---
Confusion Matrix:
    B   M
B  65   5
M   7  25
Accuracy: 0.882
Recall: 0.781
Presicion: 0.833
----------------------------------------------------------------------------------------------------
4) K-nearest Neigbors:
------------------
Confusion Matrix:
    B   M
B  66   4
M   6  26
Accuracy: 0.902
Recall: 0.812
Presicion: 0.867
----------------------------------------------------------------------------------------------------
5) GaussianNB:
----------
Confusion Matrix:
    B   M
B  65   5
M   2  30
Accuracy: 0.931
Recall: 0.938
Presicion: 0.857
----------------------------------------------------------------------------------------------------


 58%|█████▊    | 7/12 [00:00<00:00, 11.55it/s]

...

9) AdaBoost:
--------
Confusion Matrix:
    B   M
B  67   3
M   4  28
Accuracy: 0.931
Recall: 0.875
Presicion: 0.903
----------------------------------------------------------------------------------------------------


 92%|█████████▏| 11/12 [00:04<00:00,  1.44it/s]

10) XGBoost:
-------
Confusion Matrix:
    B   M
B  69   1
M   4  28
Accuracy: 0.951
Recall: 0.875
Presicion: 0.966
----------------------------------------------------------------------------------------------------


100%|██████████| 12/12 [00:07<00:00,  1.65it/s]

11) MLP:
---
Confusion Matrix:
    B   M
B  68   2
M   9  23
Accuracy: 0.892
Recall: 0.719
Presicion: 0.92
----------------------------------------------------------------------------------------------------


In [85]:
metrics_df = pd.DataFrame(lst_1, columns=['Algorithm','TN', 'FN', 'FP', 'TP', 'Accuracy','Recall', 'Precision'])
round(metrics_df, 3)

,Algorithm,Accuracy,Recall,Precision,TN,FN,FP,TP
0,Logistic Regression,0.941,0.875,0.933,68,2,4,28
1,SVM,0.931,0.812,0.963,69,1,6,26
2,Linear SVM,0.902,0.812,0.867,66,4,6,26
3,SGD,0.882,0.781,0.833,65,5,7,25
4,K-nearest Neigbors,0.902,0.812,0.867,66,4,6,26
5,GaussianNB,0.931,0.938,0.857,65,5,2,30
6,Decision Tree,0.931,0.875,0.903,67,3,4,28
7,Bagging,0.941,0.906,0.906,67,3,3,29
8,Random Forest,0.941,0.938,0.882,66,4,2,30
9,AdaBoost,0.931,0.875,0.903,67,3,4,28
